In [187]:
# imports
from copy import copy, deepcopy
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn import linear_model
import pandas as pd 
import numpy as np
from sklearn import metrics  
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.preprocessing import LabelEncoder
import random
import string

%matplotlib inline

In [188]:
# Load the Wine Dataset
df_wine = pd.read_csv("df_wine_country_typos.csv", encoding = 'utf8', index_col=0)
df_wine = df_wine.reset_index()
# check data
df_wine.head(10)

,level_0,index,country,province,region,price,variety,points,year_of_wine
0,0,0,US,Oregon,Willamette Valley,14.0,Pinot Gris,87,2013
1,1,1,US,Michigan,Lake Michigan Shore,13.0,Riesling,87,2013
2,2,2,US,Oregon,Willamette Valley,65.0,Pinot Noir,87,2012
3,3,3,Spain,Northern Spain,Navarra,15.0,Tempranillo-Merlot,87,2011
4,4,4,Italy,Sicily & Sardinia,Vittoria,16.0,Frappato,87,2013
5,5,5,France,Alsace,Alsace,24.0,Gewürztraminer,87,2012
6,6,6,France,Alsace,Alsace,27.0,Pinot Gris,87,2012
7,7,7,US,California,Napa,19.0,Cabernet Sauvignon,87,2011
8,8,8,France,Alsace,Alsace,30.0,Gewürztraminer,87,2012
9,9,9,US,California,Sonoma,34.0,Cabernet Sauvignon,87,2012


In [189]:
# check distinct countries
countries = pd.unique(df_wine['country'])
print(countries)

['US' 'Spain' 'Italy' ... 'UqSt' 'UjA' 'UZSs']


In [190]:
# todo: check frequencies of country
frequencies = df_wine['country'].value_counts()
frequencies.sort_values
print(frequencies)

US           47805
France       16319
Italy        16069
Spain         6059
Argentina     3664
Australia     2223
Canada         253
UVS             16
USS             15
USM             15
UOS             15
USG             14
UvS             14
UDS             14
USi             14
UQS             14
USY             13
UPS             13
USj             13
USf             13
UcS             13
UsS             13
UHS             13
UUS             12
UMS             12
USx             12
USl             12
UaS             12
USa             12
UKS             12
             ...  
UBSm             1
UdSC             1
AiS              1
Uwl              1
UWM              1
wkeS             1
Udg              1
UQiS             1
hKS              1
ULbS             1
UNV              1
dhiS             1
UEHD             1
YlSl             1
NNxS             1
UPSM             1
Ugw              1
UKSI             1
UMGr             1
UfP              1
UlJ              1
Uzq         

In [191]:
name = frequencies.index[0]
value = frequencies[0]
print("Most frequent: {name} ({value})".format(name=name, value=value))

Most frequent: US (47805)


# TYPOS DETECTION

In [192]:
def clean_value(value, original, percent):
    typos_chars = list(value)
    items = list(original)
    
    check_list = dict()
    for item in items:
        check_list[item] = False
    
    # check if value contains all chars of original
    for typo in typos_chars:
        if typo in original:
            if check_list[typo]:
                continue
            check_list[typo] = True
    
    # how many characters from the original were found, 
    # correctnes 100 means, all characters in the original were found
    correctness = int((percent * len(original)) / 100.0)
    corrected = len([x for x in check_list.values() if x])

    if corrected < correctness:
#         print("[ERROR] Couldnt clean: {}.".format(value))
        return value

    return original

In [193]:
def detect_typos(df, column_name, column_value, expected_distinct_values, correctness):
    i = 0
    b = 0
    print("[OK] Clean correctness: {}%".format(correctness))
    for index, row in df.iterrows():
        value = row[column_name]
        if value in expected_distinct_values:
            continue
        
        new_value = clean_value(value, column_value, correctness)
        
        if value != new_value:
            df.at[index, column_name] = new_value
#             print("[OK] Cleaning {} results in {}".format(value, new_value))
            i += 1
        else:
            b += 1
    
    print("[OK] Typos detected: {}".format(b+i))
    print("[OK] Cleaned: {} rows".format(i))
    print("[WARN] Couldn't clean: {} rows".format(b)) 
    return df

In [194]:
expected_distinct_values = ['US', 'Spain', 'Italy', 'France', 'Argentina', 'Australia', 'Canada']
correctness = 100

df_wine = detect_typos(df_wine, "country", "US", expected_distinct_values, correctness)

[OK] Clean correctness: 100%
[OK] Typos detected: 5311
[OK] Cleaned: 2362 rows
[WARN] Couldn't clean: 2949 rows


In [195]:
# check results
frequencies = df_wine['country'].value_counts()
frequencies.sort_values

print(frequencies)

US           50167
France       16319
Italy        16069
Spain         6059
Argentina     3664
Australia     2223
Canada         253
qSu              4
UHo              3
GSZ              3
UbA              3
Utz              3
Uju              3
UNC              3
UrL              3
UGx              3
Ubd              3
UHJ              3
UeP              3
VOS              3
Ucr              3
UNL              3
UtV              3
ULm              3
UNd              3
Uxf              3
Uhq              3
UAn              3
UOH              3
wtS              3
             ...  
vIGS             1
UVd              1
ZySS             1
lSD              1
UJC              1
UYDR             1
tVPS             1
EESg             1
eOsS             1
UdeW             1
slnS             1
UEHD             1
bapS             1
YlSl             1
NNxS             1
AiS              1
UMGr             1
UfP              1
UlJ              1
Uzq              1
QIS              1
Uzo         

In [196]:
# Auto encodes any dataframe column of type category or object.
def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                df[feature] = le.fit_transform(df[feature])
            except:
                print('Error encoding '+feature)
        return df

In [197]:
# one hot encoding
df_dummy = dummyEncode(df_wine[['country','province','region','variety']])
df_dummy.columns = ['country_num', 'province_num','region_num','variety_num']
df_wine = pd.concat([df_wine, df_dummy], axis=1)

/Users/roxana/.virtualenvs/dojo3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [198]:
df_wine.head(4)
# us country num = 6

,level_0,index,country,province,region,price,variety,points,year_of_wine,country_num,province_num,region_num,variety_num
0,0,0,US,Oregon,Willamette Valley,14.0,Pinot Gris,87,2013,1027,40,999,311
1,1,1,US,Michigan,Lake Michigan Shore,13.0,Riesling,87,2013,1027,27,472,340
2,2,2,US,Oregon,Willamette Valley,65.0,Pinot Noir,87,2012,1027,40,999,315
3,3,3,Spain,Northern Spain,Navarra,15.0,Tempranillo-Merlot,87,2011,485,36,629,423


In [199]:
# Variety
df_variety = df_wine.variety.str.get_dummies()
df_variety.columns = ['v_' + col for col in df_variety.columns] # naming the columns
df_wine = pd.concat([df_wine, df_variety], axis=1) # combine main dataframe with country matrix dataframe
df_wine = df_wine.drop('variety', axis=1) # drop the country attribute since different attributes per country value
df_wine = df_wine.drop('variety_num', axis=1) # drop the country attribute since different attributes per country value

KeyboardInterrupt: 

In [ ]:
# Country
df_country = df_wine.country.str.get_dummies() # get the pivot of the country attribte
df_country.columns = ['country_' + col for col in df_country.columns] # naming the columns
df_wine = pd.concat([df_wine, df_country], axis=1) # combine main dataframe with country matrix dataframe
df_wine = df_wine.drop('country', axis=1) # drop the country attribute since different attributes per country value
df_wine = df_wine.drop('country_num', axis=1) # drop the country attribute since different attributes per country value

In [ ]:
# Province
df_province = df_wine.province.str.get_dummies() # get the pivot of the country attribte
df_province.columns = ['province_' + col for col in df_province.columns] # naming the columns
df_wine = pd.concat([df_wine, df_province], axis=1) # combine main dataframe with country matrix dataframe
df_wine = df_wine.drop('province', axis=1) # drop the country attribute since different attributes per country value
df_wine = df_wine.drop('province_num', axis=1) # drop the country attribute since different attributes per country value

In [ ]:
# Region
df_wine = df_wine.drop('region', axis=1) # drop the country attribute since different attributes per country value
df_wine = df_wine.drop('region_num', axis=1) # drop the country attribute since different attributes per country value

In [ ]:
df_wine.head(5)

In [ ]:
# check new headers
headers = list(df_wine)

for header in headers:
    if "country_" in header:
        print(header)

In [ ]:
# validate number of wines from US, should be 53116
frequencies_us = df_wine['country_US'].value_counts()
if frequencies_us[1] == us_wines.shape[0]:
    print("[OK] Sizes match after hot encoding.")

In [ ]:
# check only numbers in data
df_wine.head(5)

In [ ]:
# split data
df_X = df_wine.drop('points', axis=1)
df_Y = df_wine[['points']]
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.3, random_state=0) 

In [ ]:
def get_hyperparameters(X_train, y_train):
    regressor3 = DecisionTreeRegressor()
    param_dist = {'max_depth': sp_randint(2,16),
                  'min_samples_split': sp_randint(2,16)}

    n_iter_search = 20
    clfrs = RandomizedSearchCV(regressor3, 
                               param_distributions=param_dist,
                               scoring='neg_mean_squared_error',
                               cv=5 , n_jobs=1, verbose=1,
                               n_iter=n_iter_search)
    clfrs.fit(X_train, y_train)
    params = clfrs.best_params_
    return params["max_depth"], params["min_samples_split"]

In [ ]:
# get hyperparameters
max_depth, min_samples_split = get_hyperparameters(X_train, y_train)
print("max_depth: {}, min_samples_split: {}".format(max_depth, min_samples_split)) 

# RESULT TYPOS CLEAN

In [ ]:
regressor = DecisionTreeRegressor(max_depth=max_depth, min_samples_split=min_samples_split)  

# Train the model using the training sets
regressor.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regressor.predict(X_test)

# The evaluation metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 